In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict

# Team

### Paolo Ferrari (netID: paolof2)
### Shitao Liu (netID: sl53)
### Yue Zhang (netID: yuez11)

# 1. Data prep & processing

### Read data:

In [3]:
ratings = pd.read_table("https://liangfgithub.github.io/MovieData/ratings.dat",sep="::",header=None,engine='python')
ratings.columns = ['UserID','MovieID','Rating','Timestamp']
ratings['UserID'] = ratings['UserID'].apply(lambda x:'u' + str(x))
ratings['MovieID'] = ratings['MovieID'].apply(lambda x:'m' + str(x))

In [4]:
ratings

,UserID,MovieID,Rating,Timestamp
0,u1,m1193,5,978300760
1,u1,m661,3,978302109
2,u1,m914,3,978301968
3,u1,m3408,4,978300275
4,u1,m2355,5,978824291
...,...,...,...,...
1000204,u6040,m1091,1,956716541
1000205,u6040,m1094,5,956704887
1000206,u6040,m562,5,956704746
1000207,u6040,m1096,4,956715648


In [5]:
movies = pd.read_table("https://liangfgithub.github.io/MovieData/movies.dat",sep="::",header=None,engine='python',encoding='latin-1')
movies.columns = ['MovieID','Title','Genres']
movies['MovieID'] = movies['MovieID'].apply(lambda x:'m' + str(x))
movies

,MovieID,Title,Genres
0,m1,Toy Story (1995),Animation|Children's|Comedy
1,m2,Jumanji (1995),Adventure|Children's|Fantasy
2,m3,Grumpier Old Men (1995),Comedy|Romance
3,m4,Waiting to Exhale (1995),Comedy|Drama
4,m5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,m3948,Meet the Parents (2000),Comedy
3879,m3949,Requiem for a Dream (2000),Drama
3880,m3950,Tigerland (2000),Drama
3881,m3951,Two Family House (2000),Drama


## Part 1: Recommendation based on genres

For doing recommendation based on genres, we first calculate the average rating of each movie, the number of ratings received by each movie, and the genres of each movie.


In [6]:
users = pd.read_table("https://liangfgithub.github.io/MovieData/users.dat",sep="::",header=None,engine='python',encoding='latin-1')
users.columns = ['UserID','Gender', 'Age', 'Occupation', 'Zip-code']
users['UserID'] = users['UserID'].apply(lambda x:'m' + str(x))

In [7]:
genre_dic = defaultdict(list)
for i in range(len(movies)):
    info = movies.iloc[i]
    mid = info['MovieID']
    title = info['Title']
    genres = info['Genres'].split('|')
    for genre in genres:
        genre_dic[genre].append(mid)

In [8]:
id_title_mapping = {}
for i in range(len(movies)):
    id_title_mapping[movies['MovieID'].iloc[i]] = movies['Title'].iloc[i]

In [9]:
movie_avr_score = defaultdict(float)
movie_rated_frq = defaultdict(int)

In [10]:
rating_by_movie = ratings.groupby('MovieID',group_keys=False).mean()
rating_by_movie

,Rating,Timestamp
MovieID,,
m1,4.146846,9.705586e+08
m10,3.540541,9.720168e+08
m100,3.062500,9.717403e+08
m1000,3.050000,9.716269e+08
m1002,4.250000,9.707060e+08
...,...,...
m994,4.095556,9.727394e+08
m996,2.906250,9.725015e+08
m997,3.357143,9.718940e+08


In [11]:
for idx in rating_by_movie.index:
    movie_avr_score[idx] = rating_by_movie['Rating'].loc[idx]

### Method 1: Most Popular Recommendation

The first genre-based recommendation method we propose is "most popular recommendation". Specifically, we define "populartity" by the number of ratings a movie receives. The more ratings a movie receives, the more popular it is.

To recommend the "top 5 most popular movie" for each genre, we first find all movies belonging to this genre. Then, we sort the movies by the number of ratings they receive, and return the top 5 movies as our recommendation results.

In [12]:
freq_by_movie = ratings.groupby('MovieID', group_keys=False).count()


In [13]:
for idx in freq_by_movie.index:
    movie_rated_frq[idx] = freq_by_movie['Rating'].loc[idx]

In [14]:
def Popular_Recom(fav):
    if (fav not in genre_dic.keys()):
        return ''
    similar_movies = genre_dic[fav]
    similar_movies.sort(key=lambda x: movie_rated_frq[x], reverse=True)
    return similar_movies[:5]

for genre in genre_dic.keys():
    mids = Popular_Recom(genre)
    titles = [id_title_mapping[i] for i in mids]
    print('\n', genre + ': {}'.format(titles))



 Animation: ['Toy Story (1995)', 'Who Framed Roger Rabbit? (1988)', "Bug's Life, A (1998)", 'Toy Story 2 (1999)', 'Aladdin (1992)']

 Children's: ['E.T. the Extra-Terrestrial (1982)', 'Toy Story (1995)', 'Babe (1995)', 'Wizard of Oz, The (1939)', "Bug's Life, A (1998)"]

 Comedy: ['American Beauty (1999)', 'Back to the Future (1985)', 'Men in Black (1997)', 'Shakespeare in Love (1998)', 'Princess Bride, The (1987)']

 Adventure: ['Star Wars: Episode IV - A New Hope (1977)', 'Star Wars: Episode V - The Empire Strikes Back (1980)', 'Star Wars: Episode VI - Return of the Jedi (1983)', 'Jurassic Park (1993)', 'Men in Black (1997)']

 Fantasy: ['Star Wars: Episode IV - A New Hope (1977)', 'E.T. the Extra-Terrestrial (1982)', 'Star Wars: Episode I - The Phantom Menace (1999)', 'Beetlejuice (1988)', 'Big (1988)']

 Romance: ['Star Wars: Episode VI - Return of the Jedi (1983)', 'Shakespeare in Love (1998)', 'Princess Bride, The (1987)', 'Groundhog Day (1993)', 'Forrest Gump (1994)']

 Drama: 

### Method 2: Highest Rating Recommendation

The second genre-based recommendation method we propose is "highest rated recommendation". To recommend the "top 5 highest rated movie" for each genre, we first find all movies belonging to this genre. Then, we sort the movies by the average rating they receive, and return the top 5 movies as our recommendation results.



In [15]:
def Rating_Recom(fav):
    if (fav not in genre_dic.keys()):
        return ''
    similar_movies = genre_dic[fav]
    similar_movies.sort(key=lambda x: movie_avr_score[x], reverse=True)
    return similar_movies[:5]

for genre in genre_dic.keys():
    mids = Rating_Recom(genre)
    titles = [id_title_mapping[i] for i in mids]
    print(genre + ': {}'.format(titles))
    print()

Animation: ['Close Shave, A (1995)', 'Wrong Trousers, The (1993)', 'Wallace & Gromit: The Best of Aardman Animation (1996)', 'Grand Day Out, A (1992)', 'Creature Comforts (1990)']

Children's: ['Wizard of Oz, The (1939)', 'Toy Story 2 (1999)', 'Toy Story (1995)', 'Iron Giant, The (1999)', 'Winnie the Pooh and the Blustery Day (1968)']

Comedy: ['Smashing Time (1967)', 'Follow the Bitch (1998)', 'One Little Indian (1973)', 'Close Shave, A (1995)', 'Wrong Trousers, The (1993)']

Adventure: ['Ulysses (Ulisse) (1954)', 'Sanjuro (1962)', 'Raiders of the Lost Ark (1981)', 'Star Wars: Episode IV - A New Hope (1977)', 'Lawrence of Arabia (1962)']

Fantasy: ['Star Wars: Episode IV - A New Hope (1977)', 'Hungarian Fairy Tale, A (1987)', 'E.T. the Extra-Terrestrial (1982)', 'Heavenly Creatures (1994)', 'Willy Wonka and the Chocolate Factory (1971)']

Romance: ['Skipped Parts (2000)', 'Casablanca (1942)', 'City Lights (1931)', 'Princess Bride, The (1987)', 'Philadelphia Story, The (1940)']

Drama:

## Part II: UBCF and IBCF

### User-Item Matrix

Here, we create a index-username map to map each username to a row in the user-item rating matrix


In [16]:
users_ids = sorted(list(set(ratings['UserID'].values)))


In [17]:
user_index_map = {}
index_user_map = {}
for i in range(len(users_ids)):
    user_index_map[users_ids[i]] = i
    index_user_map[i] = users_ids[i]

In [18]:
movies_ids = sorted(list(set(ratings['MovieID'].values)))


In [19]:
movie_index_map = {}
index_movie_map = {}
for i in range(len(movies_ids)):
    movie_index_map[movies_ids[i]] = i
    index_movie_map[i] = movies_ids[i]

Create the user-item grading matrix. Row: User; Col: Item


In [20]:
ui_grading_matrix = np.zeros([len(users_ids), len(movies_ids)])

for i in range(ui_grading_matrix.shape[0]):
    uid = index_user_map[i]
    user_ratings = ratings.loc[ratings['UserID'] == uid]
    for j in range(len(user_ratings)):
        msg = user_ratings.iloc[j]
        mid = msg['MovieID']
        ui_grading_matrix[i, movie_index_map[mid]] = msg['Rating']

In [21]:
iu_grading_matrix = ui_grading_matrix.copy()


Calculate the average rating for each user and normalize each row


In [22]:
user_avr = {}
for i in range(ui_grading_matrix.shape[0]):
    count = 0
    summation = 0
    for rating in ui_grading_matrix[i]:
        if (rating != 0):
            summation += rating
            count += 1
    user_avr[i] = summation / count

In [23]:
for i in range(ui_grading_matrix.shape[0]):
    for j in range(ui_grading_matrix.shape[1]):
        if (ui_grading_matrix[i, j] != 0):
            ui_grading_matrix[i, j] -= user_avr[i]

### UBCF

In [24]:
def cosine_sim(x, y):    
    x_adj = []
    y_adj = []
    for i in range(len(x)):
        if (x[i] != 0 and y[i] != 0):
            x_adj.append(x[i])
            y_adj.append(y[i])
    x = np.asarray(x_adj)
    y = np.asarray(y_adj)
    if (len(x) == 0 or len(y) == 0):
        return -float('inf'), -float('inf')
    
    return (1 + np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))) / 2, len(x)

In [25]:
def get_similarities(training_users, test_vector, threshold):
    similarity_heap = []
    for i in range(training_users.shape[0]):
        similarity, common = cosine_sim(training_users[i], test_vector)
        similarity_heap.append((similarity, i))
    similarity_heap.sort(key=lambda x: x[0])
    return similarity_heap[-threshold:]

In [26]:
def pred_rating(training_users, neighbours):
    prediction = [np.NaN for _ in range(training_users.shape[1])]
    for j in range(training_users.shape[1]):
        if (test_vector[j] != 0):
            continue
        total_weighted_rating = 0
        total_sim = 0
        for neighbour in neighbours:
            (similarity, neighbour_idx) = neighbour
            if (training_users[neighbour_idx, j] != 0):
                total_sim += similarity
                total_weighted_rating += similarity * training_users[neighbour_idx, j]
        if (total_weighted_rating != 0):
            prediction[j] = total_weighted_rating / total_sim
    return np.asarray(prediction)

We use the first 500 users as the training data, and use the 501st user as the test data. For UBCF, we only consider the top 20 most similar users.


In [27]:
test_user = 501
threshold = 20

test_vector = ui_grading_matrix[test_user - 1, :]

In [28]:
training_users = ui_grading_matrix[ : test_user - 1, :]


Calculate the cosine similarity between users and make prediction


In [29]:
similarity = get_similarities(training_users, test_vector, threshold)
prediction = pred_rating(training_users, similarity)

De-normalize by adding the average rating back


In [30]:
for idx, pred in enumerate(prediction):
    if (np.isnan(pred)):
        continue
    else:
        prediction[idx] += user_avr[test_user - 1]

### IBCF

In [32]:
new_test = iu_grading_matrix[test_user - 1, :]
ibcf_k = 30


In [33]:
def get_ibcf_similarity(training, threshold):
    movie_similarity_dic = defaultdict(list)
    training = training.T
    for i in range(training.shape[0]):
        vec1 = training[i]
        similars = []
        for j in range(training.shape[0]):
            if (i == j):
                continue
            vec2 = training[j]
            similarity, commons = cosine_sim(vec1, vec2)
            similars.append((similarity, commons, j))
        similars.sort(key=lambda x:(x[0]))
        movie_similarity_dic[i] = similars[-threshold:]
    return movie_similarity_dic

In [34]:
def IBCF_rating(movie_similarity_dic, testing_user):
    predictions = [np.NaN for _ in range(len(testing_user))]
    for j in range(len(predictions)):
        if (testing_user[j] != 0):
            continue
        total_weighted_rating = 0
        total_sim = 0
        for (similarity, commons, movie_idx) in movie_similarity_dic[j]:
            if (testing_user[movie_idx] != 0 and similarity != -float('inf')):
                total_sim += similarity
                total_weighted_rating += similarity * testing_user[movie_idx]
        if (total_weighted_rating != 0):
            predictions[j] = total_weighted_rating / total_sim
    return predictions

In [35]:
movie_similarities = get_ibcf_similarity(training_users, ibcf_k)


In [36]:
ibcf_pred = IBCF_rating(movie_similarities, new_test)
